In [2]:
from PIL import Image
import cairosvg
import os
import random

# Function to convert svg to png
def convert_svg_to_png(svg_path, png_path):
    cairosvg.svg2png(url=svg_path, write_to=png_path)

# Function to overlay images with transformations
def overlay_images_with_transformations(png_path1, png_path2, output_path, transformations):
    image1 = Image.open(png_path1).convert("RGBA")
    image2 = Image.open(png_path2).convert("RGBA")
    
    # Apply transformations
    for transform in transformations:
        if transform['type'] == 'rotate':
            image2 = image2.rotate(transform['value'], expand=True)
    
    # Ensure both images are the same size
    image2 = image2.resize(image1.size)
    
    # Calculate the bounding box of the rotated gripper
    bbox = image2.getbbox()
    max_shift_x = image1.width - (bbox[2] - bbox[0])
    max_shift_y = image1.height - (bbox[3] - bbox[1])
    
    # Apply shift transformation
    for transform in transformations:
        if transform['type'] == 'shift':
            shift_x = min(max(transform['x'], 0), max_shift_x)
            shift_y = min(max(transform['y'], 0), max_shift_y)
            image2 = image2.transform(image1.size, Image.AFFINE, (1, 0, shift_x, 0, 1, shift_y))
    
    # Overlay images
    combined = Image.alpha_composite(image1, image2)
    combined.save(output_path)

# Paths to your images
svg_file = "data/dummy/part_1/gripper_2.svg"
png_file = "data/dummy/part_1/part_1.png"
converted_png_file = "data/Converted_Images/converted_file.png"
output_folder = "data/Output_Images/"

# Create output folder if it doesn't exist
os.makedirs(output_folder, exist_ok=True)

# Convert svg to png
convert_svg_to_png(svg_file, converted_png_file)

# Generate transformations and save images
num_images = 10  # Number of images to generate
for i in range(1, num_images + 1):
    # Ensure the gripper stays within the bounds of the metal sheet
    transformations_list = [
        {'type': 'rotate', 'value': random.randint(0, 360)},
        {'type': 'shift', 'x': random.randint(-50, 50), 'y': random.randint(-50, 50)}
    ]
    
    output_file = os.path.join(output_folder, f"result_{i}.png")
    overlay_images_with_transformations(png_file, converted_png_file, output_file, transformations_list)

/tmp/ipykernel_56709/2458004634.py:33: DeprecationWarning: AFFINE is deprecated and will be removed in Pillow 10 (2023-07-01). Use Transform.AFFINE instead.
  image2 = image2.transform(image1.size, Image.AFFINE, (1, 0, shift_x, 0, 1, shift_y))
